# Modify the RGI6 regions files for RGI7 

List of changes:
- Region 12 (Caucasus and Middle East): there is a cluster of glaciers south of the current extent of the region and subregion polygons. There are no regions below them, so there shouldn't be much of an issue in simply updating the geometry a little bit: we shift the southern boundary by 2° (from 32°N to 30°N).
- the data type of the RGI_CODE attribute in the region file of version 6 is int. For consistency with the RGI files, it should be str (format with leading zero, for example `02`). We change this as well.

In [1]:
# go down from rgi7_scripts/workflow/preprocessing
data_dir = '../../../rgi7_data/'

In [2]:
import os
import numpy as np
import shapely.geometry as shpg
import geopandas as gpd
from utils import mkdir

## Regions 

In [3]:
out_dir = os.path.abspath(os.path.join(data_dir, '00_rgi70_regions'))
mkdir(out_dir)

'/home/francesc/repositories/rgi7_data/00_rgi70_regions'

In [4]:
# Read the RGI region files
rgi_dir = os.path.join(data_dir, 'l0_RGIv6')
rgi_reg = gpd.read_file('zip://' + os.path.join(data_dir, 'l0_RGIv6', '00_rgi60_regions.zip', '00_rgi60_O1Regions.shp'))

In [5]:
# Select the RGI 12 polygon
poly = rgi_reg.loc[rgi_reg.RGI_CODE == 12].iloc[0].geometry
poly.bounds

(32.0, 31.0, 54.0, 45.0)

Let's go down to 30° South instead:

In [6]:
x, y = poly.exterior.xy
ny = np.where(np.isclose(y, 31), 30, y)
new_poly = shpg.Polygon(np.array((x, ny)).T)
rgi_reg.loc[rgi_reg.RGI_CODE == 12, 'geometry'] = new_poly

In [7]:
# Change type and format
rgi_reg['RGI_CODE'] = ['{:02d}'.format(int(s)) for s in rgi_reg.RGI_CODE]

In [8]:
rgi_reg

,FULL_NAME,RGI_CODE,WGMS_CODE,geometry
0,Alaska,01,ALA,"POLYGON ((-133.00000 54.50000, -134.00000 54.5..."
1,Alaska,01,ALA,"POLYGON ((180.00000 50.00000, 179.00000 50.000..."
2,Western Canada and USA,02,WNA,"POLYGON ((-133.00000 54.50000, -132.00000 54.5..."
3,"Arctic Canada, North",03,ACN,"POLYGON ((-125.00000 74.00000, -125.00000 75.0..."
4,"Arctic Canada, South",04,ACS,"POLYGON ((-90.00000 74.00000, -89.00000 74.000..."
5,Greenland Periphery,05,GRL,"POLYGON ((-75.00000 77.00000, -74.73000 77.510..."
6,Iceland,06,ISL,"POLYGON ((-26.00000 59.00000, -26.00000 60.000..."
7,Svalbard and Jan Mayen,07,SJM,"POLYGON ((-10.00000 70.00000, -10.00000 71.000..."
8,Scandinavia,08,SCA,"POLYGON ((4.00000 70.00000, 4.00000 71.00000, ..."
9,Russian Arctic,09,RUA,"POLYGON ((35.00000 70.00000, 35.00000 71.00000..."


In [9]:
rgi_reg.to_file(os.path.join(out_dir, '00_rgi70_O1Regions.shp'))

In [10]:
# Check
rgi_reg = gpd.read_file(os.path.join(out_dir, '00_rgi70_O1Regions.shp'))
assert rgi_reg.RGI_CODE.dtype == 'O'

## Subregions 

In [11]:
rgi_reg = gpd.read_file('zip://' + os.path.join(data_dir, 'l0_RGIv6', '00_rgi60_regions.zip', '00_rgi60_O2Regions.shp'))

In [12]:
poly = rgi_reg.loc[rgi_reg.RGI_CODE == '12-02'].iloc[0].geometry
poly.bounds

(34.0, 32.0, 53.0, 42.0)

In [13]:
x, y = poly.exterior.xy
ny = np.where(np.isclose(y, 32), 30, y)
new_poly = shpg.Polygon(np.array((x, ny)).T)
rgi_reg.loc[rgi_reg.RGI_CODE == '12-02', 'geometry'] = new_poly

In [14]:
rgi_reg.to_file(os.path.join(out_dir, '00_rgi70_O2Regions.shp'))

In [15]:
rgi_reg

,FULL_NAME,RGI_CODE,WGMS_CODE,geometry
0,N Alaska,01-01,ALA-01,"POLYGON ((-169.00000 64.00000, -169.00000 65.0..."
1,Alaska Ra (Wrangell/Kilbuck),01-02,ALA-02,"POLYGON ((-144.40436 61.48531, -144.48100 61.5..."
2,Alaska Pena (Aleutians),01-03,ALA-03,"POLYGON ((-180.00000 57.00000, -179.00000 57.0..."
3,Alaska Pena (Aleutians),01-03,ALA-03,"POLYGON ((180.00000 50.00000, 179.00000 50.000..."
4,W Chugach Mtns (Talkeetna),01-04,ALA-04,"POLYGON ((-151.32973 58.74602, -151.57953 58.8..."
...,...,...,...,...
88,NE Antarctic Pena 7I2,19-21,ANT-21,"POLYGON ((-62.41123 -69.50624, -62.41913 -69.5..."
89,SE Antarctic Pena 7I3,19-22,ANT-22,"POLYGON ((-50.00000 -74.80000, -51.00000 -74.8..."
90,Ronne-Filchner Ice Shelf 7J,19-23,ANT-23,"POLYGON ((-42.00000 -74.80000, -41.00000 -74.8..."
91,W Queen Maud Land 7K,19-24,ANT-24,"POLYGON ((-10.30000 -69.40000, -10.00000 -69.4..."
